## Simple pipeline run

We'll setup a simple pipeline and use the `rubicon`'s sklearn integration to automatically log parameters and metrics as we fit and score the model.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from rubicon import Rubicon
from rubicon.sklearn import RubiconPipeline

rubicon = Rubicon(persistence="filesystem", root_dir="./rubicon-root")
project = rubicon.get_or_create_project("Rubicon Pipeline Example")

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

pipe = RubiconPipeline([('scaler', StandardScaler()), ('svc', SVC())], project)
pipe

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

### Fetch the results from Rubicon

During the pipeline run, an experiment was automatically created and the corresponding parameters and metrics logged to it. We can use the `rubicon` library to pull these experiments back or view them on the dashboard.

In [ ]:
experiments = project.experiments()
print(f"{len(experiments)} experiment(s)")
experiment = experiments[0]
experiment

In [ ]:
for param in experiment.parameters():
    print(f"{param.name}: {param.value}")

In [ ]:
for metric in experiment.metrics():
    print(f"{metric.name}: {metric.value}")

In [ ]:
! rubicon ui --root-dir "./rubicon-root"

## Use GridSearch

In [ ]:
from pprint import pprint
from time import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from rubicon import Rubicon
from rubicon.sklearn import RubiconPipeline

rubicon = Rubicon(persistence="filesystem", root_dir="./rubicon-root")
project = rubicon.get_or_create_project("Rubicon Grid Search Example")

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

pipeline = RubiconPipeline([('scaler', StandardScaler()), ('svc', SVC())], project)
pipeline

In [ ]:
parameters = {
    'scaler__with_mean': (True, False),
    'svc__kernel': ('linear', 'poly')
}

grid_search = GridSearchCV(pipeline, parameters)

print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()

grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
experiments = project.experiments()
print(f"{len(experiments)} experiments logged")

for e in experiments:
    print(e.parameters())

In [ ]:
! rubicon ui --root-dir "./rubicon-root"